## Pasamos a la fase de aprendizaje

El problema que tenemos que resolver consiste en determinar si un Pokémon tiene muchas posibilidades de ganar durante un combate.

Como disponemos de datos de aprendizaje, entonces estamos en un caso de Machine Learning denominado «supervisado». Es decir, que la máquina va a aprender en función de lo que se le proporciona como entrada.

En este tipo de casos disponemos de dos tipos de algoritmos: los dedicados a la clasificación o los dedicados a la regresión.

La clasificación permite organizar las predicciones en grupo, mientras que la regresión permite definir un valor.

En nuestro caso tenemos que predecir el porcentaje de victoria, que es un valor, y obviamente utilizaremos los algoritmos de regresión, que son los siguientes: 

*    La regresión lineal.

*    Los árboles de decisiones.

*    Los bosques aleatorios.


La primera etapa antes de cualquier aprendizaje es distribuir las observaciones de las que disponemos en un conjunto de aprendizaje con el que aprenderá la máquina y un conjunto de pruebas con el que la máquina evaluará lo aprendido. Para eso, vamos a utilizar el módulo Scikit-Learn.

Para evitar ejecutar las distintas tareas de análisis de datos y de preparación de estos antes de cada aprendizaje, hemos guardado el **Dataframe** en un archivo llamado dataset.csv con el tabulador como separador **(sep=’\t’)**.

In [37]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Uso del módulo Pandas
import pandas as pnd

#Carga de dataset
dataset = pnd.read_csv("datas/dataset.csv",delimiter='\t')

#Eliminación de los valores NA (columnas: Primer Pokemon, Segundo Pokemon)
dataset = dataset.dropna(axis=0, how='any')


#X = se toman todos los datos, pero solo de las columnas 4 a la 11
#    PUNTOS_ATAQUE;PUNTOS_DEFENSA;PUNTOS_ATAQUE_ESPECIAL;PUNTO_DEFENSA_ESPECIAL;PUNTOS_VELOCIDAD;CANTIDAD_GENERACIONES
X = dataset.iloc[:, 5:12].values

#y = solo se toma la columna PORCENTAJE_DE_VICTORIA (16º valor)
y = dataset.iloc[:, 17].values


#Construccón del conjunto de entrenamiento y del conjunto de prueba
from sklearn.model_selection import train_test_split
X_APRENDIZAJE, X_VALIDACION, Y_APRENDIZAJE, Y_VALIDACION = train_test_split(X, y, test_size = 0.2, random_state = 0)


A partir de aqui, vamos a enseñarle a la máquina a predecir un porcentaje de victoria de un Pokémon en función de sus puntos de ataque (estándar y especiales), de su velocidad, de sus puntos de vida, de sus puntos de defensa (estándar y especiales) y de su cantidad de generaciones. Acabamos de definir, pues, las variables predictivas (X) y la variable que hay que predecir (Y).

A continuación, cada variable será distribuida en un grupo de aprendizaje (Train) y en un grupo de prueba (Test). Entonces tendremos un grupo X_APRENDIZAJE, Y_APRENDIZAJE que permite que la máquina aprenda y un grupo X_VALIDACION,Y_VALIDACION que le permite validar su aprendizaje.

Esto se hace a con la función **train_test_split** del modulo Scikit-Learn, el cual permite crear conjuntos de datos de aprendizaje y de pruebas.

Tambien es importante mencionar que la funcion **iloc** permite seleccionar las características por su numero de indice (o de columna del dataframe), y que estamos usando el 20% del dataset para el conjunto de pruebas a partir del valor del **test_size** que es de **0.2**

### 1. Algoritmo de regresión lineal

El algoritmo de regresión lineal modeliza las relaciones entre una variable predictiva y una variable de destino.

En nuestro caso, las variables predictivas son:

*    los puntos de vida,
*    el nivel de ataque,
*    el nivel de defensa,
*    el nivel de ataque especial,
*    el nivel de defensa especial,
*    la velocidad y la generación del Pokémon.

Y la variable de destino (la predicción) es:

+    el porcentaje de victoria.

En nuestro caso, tenemos varias variables predictivas para una predicción. Por lo tanto, estamos en el caso de una regresión lineal múltiple. A continuación, usamos el algoritmo de regresión lineal para hacer nuestro aprendizaje con el módulo Scikit-Learn:

In [38]:
#---- ALGORITMO 1: REGRESION LINEAL -----
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression

#Elección del algoritmo
algoritmo = LinearRegression()

#Aprendizaje con ayuda de la función fit
algoritmo.fit(X_APRENDIZAJE, Y_APRENDIZAJE)

#Realización de la predicción sobre el conjunto de prueba
predicciones = algoritmo.predict(X_VALIDACION)

#Cálculo de la precisión del aprendizaje con ayuda de la
#función r2_score
precision = r2_score(Y_VALIDACION, predicciones)


print(">> ----------- REGRESION LINEAL -----------")
print(">> Precisión = "+str(precision))
print("------------------------------------------")


>> ----------- REGRESION LINEAL -----------
>> Precisión = 0.9043488485570964
------------------------------------------


El cálculo de la precisión de aprendizaje (accuracy) se lleva a cabo sobre las predicciones realizadas a partir de los datos de pruebas. En efecto, queremos validar el nivel de eficacia del algoritmo. Para eso, hay que medirlo sobre los datos que no conoce. Hay que señalar que a veces también es interesante calcular la precisión con los datos de aprendizaje y compararla con la obtenida con los datos de pruebas, con la finalidad de determinar si se ha producido un sobreajuste.


Para medir esta precisión (también llamada score), utilizamos el método r2_score, como recomienda la documentación del módulo SciKit-Learn en el caso de problemas de regresión (https://scikit-learn.org/stable/modules/model_evaluation.html):

![Alt text](../Pokemon/images/image1.PNG)


Por el momento vamos a dejar de lado los otros indicadores de medida, especialmente la MSE (Mean Square Error), esto se verá a detalle en conceptos de redes neuronales.

A continuación, se hacen predicciones sobre los datos de pruebas (X_PRUEBA) y se comparan con las esperadas (Y_PRUEBA) usando el método r2_Score encargado de calcular la precisión del algoritmo.

Esto nos permite deducir que, con la regresión lineal múltiple, obtenemos un score del 90 %, lo que es bastante bueno.

>> ----------- REGRESION LINEAL -----------  
>> Precision = 0.9043488485570964 

### 2. El árbol de decisión aplicado a la regresión

Ahora vamos a utilizar árboles de decisión prestando atención a hacer un buen uso del algoritmo destinado a los problemas de regresión: DecisionTreeRegressor, y no al destinado a los problemas de clasificación: DecisionTreeClassifier.

In [39]:
#---- ALGORITMO 2: ARBOL DE DECISION APLICA A LA REGRESION-----


#Elección del algoritmo
from sklearn.tree import DecisionTreeRegressor
algoritmo = DecisionTreeRegressor()

#Aprendizaje con ayuda de la función fit
algoritmo.fit(X_APRENDIZAJE, Y_APRENDIZAJE)

#Realización de la predicción sobre el conjunto de prueba
predicciones = algoritmo.predict(X_VALIDACION)

#Cálculo de la precisión del aprendizaje con ayuda de la
#función r2_score
precision = r2_score(Y_VALIDACION, predicciones)


print(">> ----------- ARBOLES DE DECISION -----------")
print(">> Precisión = "+str(precision))
print("------------------------------------------")


>> ----------- ARBOLES DE DECISION -----------
>> Precisión = 0.883457661041538
------------------------------------------


### 3. Random forest

Ahora vamos a probar el algoritmo de los bosques aleatorios (siempre aplicado a la regresión):

In [40]:

#Elección del algoritmo
from sklearn.ensemble import RandomForestRegressor
algoritmo = RandomForestRegressor()

#Aprendizaje con la ayuda de la función fit
algoritmo.fit(X_APRENDIZAJE, Y_APRENDIZAJE)

#Realización de la predicción sobre el conjunto de prueba
predicciones = algoritmo.predict(X_VALIDACION)

#Cálculo de la precisión del aprendizaje con ayuda de la
#función r2_score
precision = r2_score(Y_VALIDACION, predicciones)


print(">> ----------- ARBOLES ALEATORIOS -----------")
print(">> Precisión = "+str(precision))
print("------------------------------------------")


>> ----------- ARBOLES ALEATORIOS -----------
>> Precisión = 0.9370571629540589
------------------------------------------


Hasta aqui hemos encontrado resultados satisfactorios con estos algoritmos. Sin embargo, hay de dejar claro que aquí utilizamos los algoritmos en su forma más sencilla, es decir, sin optimizar sus parámetros. Es posible que torturándolos un poco obtuviéramos otros resultados, quizás incluso más prometedores, con uno de los tres algoritmos. 

### 4. Guardar el modelo de aprendizaje

Para poder utilizar el modelo de predicción recién probado en una aplicación sin tener que repetir la fase de aprendizaje, es preciso guardarlo.

Para hacerlo, después de haber creado un directorio modelo nuevo en nuestro proyecto, vamos a utilizar el submódulo joblib del módulo Scikit-Learn para guardar nuestro modelo de aprendizaje.

In [41]:
#Guardar el algoritmo
import joblib
archivo = 'modelo/modelo_pokemon.mod'
joblib.dump(algoritmo, archivo)

['modelo/modelo_pokemon.mod']


## Fenómenos de sobreajuste (overfitting)  y de subajuste (underfitting)

Cuando buscamos una solución a un problema en Machine Learning, queremos que se pueda generalizar.

Es decir, que la solución encontrada debe ser aplicable a datos desconocidos por las soluciones utilizadas durante el aprendizaje. Para conseguirlo necesitamos que los datos utilizados durante el aprendizaje sean lo más cercanos posible a la realidad del problema que hay que resolver.

El sobreajuste (overfitting) es un fenómeno que se manifiesta por el hecho de que la solución está demasiado adaptada a los datos de aprendizaje y no se puede generalizar a datos nuevos que le son desconocidos. Así, si para un algoritmo obtenemos una precisión del 99 % sobre los datos de aprendizaje y un valor del 20 % sobre los datos de pruebas, hay muchas posibilidades de que estemos ante un sobreajuste. Por eso se recomienda medir la precisión a la vez sobre los datos de aprendizaje y los datos de validación:

>>predicciones = algoritmo.predict(X_VALIDACION)  

>>**precision_aprendizaje =** 
>>**algoritmo.score(X_APRENDIZAJE,Y_APRENDIZAJE)**

>>precision = r2_score(Y_VALIDACION, predicciones) 

En cuanto al fenómeno del subajuste, este último se produce cuando el algoritmo no llega a encontrar una relación entre los datos de aprendizaje y, por lo tanto, no consigue hacer buenas predicciones.

A menudo, nos encontramos con el concepto de overfitting durante el uso de las redes neuronales. De hecho, durante el aprendizaje buscamos minimizar los errores de predicciones realizando una determinada cantidad de bucles de aprendizaje (iteraciones). El algoritmo aprenderá de sus errores con cada bucle y se corregirá.

Cuantas más iteraciones hagamos, más aprende el algoritmo y menos se equivoca. Si se hacen demasiadas iteraciones, el algoritmo tendrá una precisión muy buena sobre los datos de aprendizaje, pero se especializará en ellos y tendrá mala precisión sobre los datos de pruebas. Por lo tanto, la generalización ya no es posible.


## Utilizar el modelo de aprendizaje en una aplicación

Disponemos de un modelo de aprendizaje capaz de predecir el porcentaje de victoria de cada Pokémon. Ahora vamos a crear una aplicación utilizando este modelo de aprendizaje con el objetivo de predecir el vencedor de un combate en el que se enfrentarán dos Pokémon elegidos en el Pokédex.

Primero empezamos con importar los modulos necesarios.

In [1]:
#Módulo de lectura de archivos CSV
import csv

#Modulo de carga del modelo de aprendizaje
import joblib

A continuación, vamos a escribir una primera función que se encargará de buscar información sobre un Pokémon en el Pokédex a partir de su número y que será útil para nuestro modelo de predicción.

In [2]:

def busquedaInformacionPokemon(numPokemon,Pokedex):
    infosPokemon = []
    for pokemon in Pokedex:
        if (int(pokemon[0])==numPokemon):
            infosPokemon = [pokemon[0],pokemon[1],pokemon[4],pokemon[5],pokemon[6],pokemon[7],pokemon[8],pokemon[9],pokemon[10]]
            break
    return infosPokemon


Primero se crea una lista **(infoPokemon)** que permite almacenar toda la información necesaria sobre el Pokémon para la predicción de su victoria o de su derrota. A continuación, buscamos en Pokédex un Pokémon que tenga el mismo número que el que buscamos. Es importante destacar que el número del Pokémon correspondiente a la primera columna del Pokédex se ha transformado en entero **(int(pokemon[0])** para poder compararlo con el número del Pokémon que buscamos y comunicárselo a la función en forma de parámetro.

Si durante el bucle de recorrido de Pokédex el número leído corresponde al número de Pokémon buscado, entonces se extrae la información necesaria para poder reenviarla después a la función que ha llamado a nuestra función de búsqueda.

A continuación, viene la función principal de nuestra aplicación, que permite predecir cuál de los dos Pokémon que combaten será el vencedor.

In [3]:


def prediccion (numeroPokemon1, numeroPokemon2,Pokedex):
    pokemon1 = busquedaInformacionPokemon(numeroPokemon1, Pokedex)
    pokemon2 = busquedaInformacionPokemon(numeroPokemon2, Pokedex)
    modelo_prediccion = joblib.load('modelo/modelo_pokemon.mod')
    prediccion_Pokemon_1 = modelo_prediccion.predict([[pokemon1[2],pokemon1[3],pokemon1[4],pokemon1[5],pokemon1[6],pokemon1[7],pokemon1[8]]])
    prediccion_Pokemon_2 = modelo_prediccion.predict([[pokemon2[2], pokemon2[3], pokemon2[4], pokemon2[5], pokemon2[6], pokemon2[7], pokemon2[8]]])
    print ("COMBATE QUE ENFRENTA: ("+str(numeroPokemon1)+") "+pokemon1[1]+" a ("+str(numeroPokemon2)+") "+pokemon2[1])
    print ("   "+pokemon1[1]+": "+str(prediccion_Pokemon_1[0]))
    print("   " +pokemon2[1] + ": " + str(prediccion_Pokemon_2[0]))
    print ("")
    if prediccion_Pokemon_1>prediccion_Pokemon_2:
        print(pokemon1[1].upper()+" ES EL GANADOR")
    else:
        print(pokemon2[1].upper() + " ES EL GANADOR")




Esta función toma como parámetros:

*    el número del primer Pokémon,
*    el número del segundo Pokémon,
*    el Pokédex.

La primera acción de nuestra función es buscar la información sobre los dos Pokémon.

A continuación, viene la carga del modelo y la predicción para cada Pokémon:

```python
modelo_prediccion = joblib.load('modelo/modelo_pokemon.mod')  
modelo_prediccion.predict([[pokemon1[2],pokemon1[3],pokemon1[4], 
pokemon1[5],pokemon1[6],pokemon1[7],pokemon1[8]]]) 
```

Después se compara el porcentaje de victoria predicho para deducir el ganador del combate.

Al fin llega el momento de introducir las últimas líneas de programa, cambiando el archivo Pokédex, excluyendo su primera línea -que contiene los nombres de las distintas columnas (next(pokedex))- y llamando a la función de predicción.



In [7]:
#Carga de Pokedex e inicio de un combate
with open("datas/pokedex.csv", newline='') as csvfile:
    pokedex = csv.reader(csvfile)
    next(pokedex)
    prediccion(159,268,pokedex)


COMBATE QUE ENFRENTA: (159) Moltres a (268) Tyranitar
   Moltres: 0.713090155504258
   Tyranitar: 0.43033775902669114

MOLTRES ES EL GANADOR


Ahora puede predecir el ganador de todos los combates de Pokémon.

Si queremos, podemos usar el archivo **combates.csv** como fuente de verificación de una buena predicción.

# Conclusión y comentarios finales.

Ahora ya estamos al final del caso de estudio dedicado a los Pokémon. 
* La primera parte nos permitió descubrir las etapas de preparación de los datos para la resolución de un problema relacionado con Machine Learning supervisado.

* En la segunda parte hemos estudiado detalladamente el análisis más fino de los datos para poder determinar los que tienen una incidencia alta (relación) sobre la resolución de nuestro problema. Aqui es cuando se formulan hipotesis.

* Finalmente, hemos probado varios algoritmos de predicción relacionados con la regresión (porque buscábamos predecir un valor) para determinar el que nos permitirá obtener un modelo de predicción fiable. Después hemos utilizado este modelo en una aplicación.

Lo que hay que recordar de esta experiencia es que la preparación y el análisis de los datos es una fase, quizás la más importante de un proyecto de Machine Learning. Los datos bien preparados y de buena calidad permitirán hacer buenas predicciones.

Hay que recordar también los conceptos de sobreajuste y subajuste, ya que nos serán útiles en nuestras búsquedas de algoritmos que puedan responder a los problemas de Machine Learning. Un buen algoritmo es el que produce resultados que se pueden generalizar.